## overview

In [1]:
# default package
import logging
import sys 
import os
import pathlib
import IPython
import random

In [2]:
# third party package
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import psycopg2
import sqlalchemy
from sqlalchemy.sql.expression import column,select,table
from sqlalchemy import create_engine
import sklearn.datasets

In [3]:
# my package
sys.path.append(os.path.join(pathlib.Path().resolve(),"../"))

In [4]:
# reload settings
%load_ext autoreload
%autoreload 2

In [5]:
# logger
logger=logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [6]:
## grap setting
sns.set()

In [7]:
## db setting
engine = create_engine('postgresql://postgres:postgres@db:5432/postgres')
with engine.connect() as conn:
    print(conn.closed)

False


## sql to python

In [8]:
def ct(file:str,chap_num:int,num:int,sub_num:int=None,engine:sqlalchemy.engine=engine)->None:
    """
    pubulic schema内のtableをすべて削除してから、指定のtable作成
    """
    if sub_num==None: 
        filename=f"../code-ssql2/ddl/ssql2-{file}{chap_num:02}_{num:02}"
    else:
        filename=f"../code-ssql2/ddl/ssql2-{file}{chap_num:02}_{num:02}_{sub_num}"
        
    with open(filename,"r") as f:
        stmt=f.read() 
    with engine.connect() as conn:
        conn.execute("drop schema public cascade; create schema public;")
        conn.execute(stmt)

In [9]:
def exe(stmt:str,engine:sqlalchemy.engine=engine):
    """
    sql実行
    """
    with engine.connect() as conn:
        query=conn.execute(stmt)
        try:
            result=query.fetchall()
            return result
        except:
            return None

## List

In [10]:
ct("List",1,1)
exe("select 出金額 from 家計簿")

[(380,), (0,), (2800,), (5000,), (7560,)]

In [11]:
ct("List",1,2)
exe("select 日付,出金額 from 家計簿")

[(datetime.date(2018, 2, 3), 380),
 (datetime.date(2018, 2, 10), 0),
 (datetime.date(2018, 2, 11), 2800),
 (datetime.date(2018, 2, 14), 5000),
 (datetime.date(2018, 2, 18), 7560)]

In [12]:
ct("List",1,3)
exe("select * from 家計簿")

[(datetime.date(2018, 2, 3), '食費', 'コーヒーを購入', 0, 380),
 (datetime.date(2018, 2, 10), '給料', '1月の給料', 280000, 0),
 (datetime.date(2018, 2, 11), '教養娯楽費', '書籍を購入', 0, 2800),
 (datetime.date(2018, 2, 14), '交際費', '同期会の会費', 0, 5000),
 (datetime.date(2018, 2, 18), '水道光熱費', '1月の電気代', 0, 7560)]

In [10]:
ct("List",1,4)
exe("""select * from 家計簿 where 出金額>3000""")

[(datetime.date(2018, 2, 14), '交際費', '同期会の会費', 0, 5000),
 (datetime.date(2018, 2, 18), '水道光熱費', '1月の電気代', 0, 7560)]

In [14]:
ct("List",1,5)
exe("""insert into 家計簿 values ('2018-02-25','居住費','3月の家賃',0,85000)""")
exe("select * from 家計簿")

[(datetime.date(2018, 2, 3), '食費', 'コーヒーを購入', 0, 380),
 (datetime.date(2018, 2, 10), '給料', '1月の給料', 280000, 0),
 (datetime.date(2018, 2, 11), '教養娯楽費', '書籍を購入', 0, 2800),
 (datetime.date(2018, 2, 14), '交際費', '同期会の会費', 0, 5000),
 (datetime.date(2018, 2, 18), '水道光熱費', '1月の電気代', 0, 7560),
 (datetime.date(2018, 2, 25), '居住費', '3月の家賃', 0, 85000)]

In [15]:
ct("List",1,6)
exe("""update 家計簿 set 出金額=90000 where 日付='2018-02-25'""")
exe("select * from 家計簿")

[(datetime.date(2018, 2, 3), '食費', 'コーヒーを購入', 0, 380),
 (datetime.date(2018, 2, 10), '給料', '1月の給料', 280000, 0),
 (datetime.date(2018, 2, 11), '教養娯楽費', '書籍を購入', 0, 2800),
 (datetime.date(2018, 2, 14), '交際費', '同期会の会費', 0, 5000),
 (datetime.date(2018, 2, 18), '水道光熱費', '1月の電気代', 0, 7560),
 (datetime.date(2018, 2, 25), '居住費', '3月の家賃', 0, 90000)]

In [16]:
ct("List",1,7)
exe("""delete from 家計簿 where 日付='2018-02-25'""")
exe("select * from 家計簿")

[(datetime.date(2018, 2, 3), '食費', 'コーヒーを購入', 0, 380),
 (datetime.date(2018, 2, 10), '給料', '1月の給料', 280000, 0),
 (datetime.date(2018, 2, 11), '教養娯楽費', '書籍を購入', 0, 2800),
 (datetime.date(2018, 2, 14), '交際費', '同期会の会費', 0, 5000),
 (datetime.date(2018, 2, 18), '水道光熱費', '1月の電気代', 0, 7560)]

## Q

In [17]:
ct("Q",1,3,1)
exe("select * from 家計簿 where 入金額=50000")

[]

In [18]:
ct("Q",1,3,2)
exe("delete from 家計簿 where 出金額>4000")
exe("select * from 家計簿")

[(datetime.date(2018, 2, 3), '食費', 'コーヒーを購入', 0, 380),
 (datetime.date(2018, 2, 10), '給料', '1月の給料', 280000, 0),
 (datetime.date(2018, 2, 11), '教養娯楽費', '書籍を購入', 0, 2800)]

In [19]:
ct("Q",1,3,3)
exe("update 家計簿 set メモ='カフェラテ購入' where 日付='2018-02-03'")
exe("select * from 家計簿")

[(datetime.date(2018, 2, 10), '給料', '1月の給料', 280000, 0),
 (datetime.date(2018, 2, 11), '教養娯楽費', '書籍を購入', 0, 2800),
 (datetime.date(2018, 2, 14), '交際費', '同期会の会費', 0, 5000),
 (datetime.date(2018, 2, 18), '水道光熱費', '1月の電気代', 0, 7560),
 (datetime.date(2018, 2, 3), '食費', 'カフェラテ購入', 0, 380)]